# Analysis of the Results

## Setup

In [1]:
# Libraries
import pandas as pd
import numpy as np

In [3]:
# Import data
df = pd.read_csv('../predictions/Random_Forest_with_rebalancing.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6541795 entries, 0 to 6541794
Data columns (total 21 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   ANO_SID                 float64
 1   CORPORATE_DEVISION      object 
 2   Bundesland              object 
 3   Typ                     object 
 4   ORTPLZ                  int64  
 5   CONSTRACTION_DESIGN     object 
 6   CONSTRUCTION_YEAR       float64
 7   WFL                     float64
 8   ZONE                    float64
 9   TYPE_OF_DEDUCTIBLE      int64  
 10  DRAIN_PIPE_INSURED      int64  
 11  PRODUCTLINE             object 
 12  PRIOR_DAMAGES           int64  
 13  UVV-KZ                  int64  
 14  UNDERWRITER             object 
 15  YEAR                    int64  
 16  DAMAGE_HEAVY_RAIN_ZONE  float64
 17  LONGITUDE               float64
 18  LATITUDE                float64
 19  DAMAGE                  int64  
 20  Probability             float64
dtypes: float64(8), int64(7), object

In [5]:
# There are no nas in the dataset
# df.isna().sum()

In [8]:
print('Number of unique values:')
df.nunique()

Number of unique values:


ANO_SID                   1033769
CORPORATE_DEVISION              2
Bundesland                     16
Typ                             2
ORTPLZ                       6147
CONSTRACTION_DESIGN            10
CONSTRUCTION_YEAR             497
WFL                           895
ZONE                            9
TYPE_OF_DEDUCTIBLE              1
DRAIN_PIPE_INSURED              1
PRODUCTLINE                     6
PRIOR_DAMAGES                  10
UVV-KZ                          2
UNDERWRITER                     2
YEAR                           10
DAMAGE_HEAVY_RAIN_ZONE        956
LONGITUDE                  369243
LATITUDE                   369134
DAMAGE                          2
Probability               3175984
dtype: int64

In [11]:
print(df['CORPORATE_DEVISION'].unique())
print(df['TYPE_OF_DEDUCTIBLE'].unique())

['VHV' 'VGV']
[0]
